In [1]:
import pandas as pd
import os
import scipy.stats as stats
import scikit_posthocs as sp



In [7]:
path = '/export/usuarios_ml4ds/ammesa/Data/4_indexed_data'
df_weighted = pd.read_csv(os.path.join(path, 'metrics_conf_ints.csv'))
df_re = pd.read_parquet(os.path.join(path, 'full_metrics.parquet'))

In [8]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,hit_3,rank_hit_3,mrr_5,precision_5,recall_5,ndcg_5,...,mrr_10,precision_10,recall_10,ndcg_10,hit_10,rank_hit_10,time,retrieval_method,thr,weighted
0,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.142857,0.1,1.0,0.333333,1,0.142857,0.414289,TB_ENN,0,True
1,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.166667,0.1,1.0,0.356207,1,0.166667,0.415068,TB_ENN,0,True
2,0.500000,0.333333,1.0,0.63093,1,0.500000,0.500000,0.2,1.0,0.630930,...,0.500000,0.1,1.0,0.630930,1,0.500000,0.234234,TB_ENN,0,True
3,0.333333,0.333333,1.0,0.50000,1,0.333333,0.333333,0.2,1.0,0.500000,...,0.333333,0.1,1.0,0.500000,1,0.333333,0.474276,TB_ENN,0,True
4,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.100000,0.1,1.0,0.289065,1,0.100000,0.212600,TB_ENN,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533771,0.000000,0.000000,0.0,0.00000,0,0.000000,0.200000,0.2,1.0,0.386853,...,0.200000,0.1,1.0,0.386853,1,0.200000,0.009868,ANN,1,True
2533772,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0,0.000000,0.009908,ANN,1,True
2533773,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.166667,0.1,1.0,0.356207,1,0.166667,0.009971,ANN,1,True
2533774,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.125000,0.1,1.0,0.315465,1,0.125000,0.010311,ANN,1,True


In [9]:
def get_significance_marker( p_value):
    if p_value < 0.001:
        return "***"
    elif p_value < 0.01:
        return "**"
    elif p_value < 0.05:
        return "*"
    else:
        return ""
    
def format_metric(metric):
    return (
            "MRR@3" if metric == "mrr_3" else
            "MRR@5" if metric == "mrr_5" else
            "NDCG@3" if metric == "ndcg_3" else
            "NDCG@5" if metric == "ndcg_5" else
            "Precision@3" if metric == "precision_3" else
            "Precision@5" if metric == "precision_5" else
            "Recall@3" if metric == "recall_3" else
            "Recall@5" if metric == "recall_5" else
            "RHit@3" if metric == "rank_hit_3" else
            "RHit@5" if metric == "rank_hit_5" else
            "Time (s)" if "time" in metric else
            metric  # Fallback si no está en la lista
        )
        

def check_normality( values):
    stat, p_value = stats.shapiro(values)
    return p_value > 0.05  

def perform_anova( groups):
    f_stat, p_value = stats.f_oneway(*groups)
    return f_stat, p_value

def perform_kruskal_wallis( groups):
    h_stat, p_value = stats.kruskal(*groups)
    return h_stat, p_value

# Normality tests:

In [10]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,hit_3,rank_hit_3,mrr_5,precision_5,recall_5,ndcg_5,...,mrr_10,precision_10,recall_10,ndcg_10,hit_10,rank_hit_10,time,retrieval_method,thr,weighted
0,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.142857,0.1,1.0,0.333333,1,0.142857,0.414289,TB_ENN,0,True
1,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.166667,0.1,1.0,0.356207,1,0.166667,0.415068,TB_ENN,0,True
2,0.500000,0.333333,1.0,0.63093,1,0.500000,0.500000,0.2,1.0,0.630930,...,0.500000,0.1,1.0,0.630930,1,0.500000,0.234234,TB_ENN,0,True
3,0.333333,0.333333,1.0,0.50000,1,0.333333,0.333333,0.2,1.0,0.500000,...,0.333333,0.1,1.0,0.500000,1,0.333333,0.474276,TB_ENN,0,True
4,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.100000,0.1,1.0,0.289065,1,0.100000,0.212600,TB_ENN,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533771,0.000000,0.000000,0.0,0.00000,0,0.000000,0.200000,0.2,1.0,0.386853,...,0.200000,0.1,1.0,0.386853,1,0.200000,0.009868,ANN,1,True
2533772,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0,0.000000,0.009908,ANN,1,True
2533773,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.166667,0.1,1.0,0.356207,1,0.166667,0.009971,ANN,1,True
2533774,0.000000,0.000000,0.0,0.00000,0,0.000000,0.000000,0.0,0.0,0.000000,...,0.125000,0.1,1.0,0.315465,1,0.125000,0.010311,ANN,1,True


In [12]:
    
df_weighted["BestValue"] = df_weighted.groupby(["FileID", "Metric"])["Weighted Mean"].transform(
    lambda x: x.min() if "Time (s)" in x.name else x.max()
)
df_weighted

,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,10_04_25_questions,TB_ENN,mrr@3,0.150019,0.001282,0.150019
1,10_04_25_questions,TB_ENN,mrr@5,0.185012,0.001255,0.185012
2,10_04_25_questions,TB_ENN,mrr@10,0.235652,0.001145,0.235652
3,10_04_25_questions,TB_ENN,precision@3,0.080673,0.000609,0.080673
4,10_04_25_questions,TB_ENN,precision@5,0.079525,0.000418,0.079525
...,...,...,...,...,...,...
223,10_04_25_questions,ANN,hit@10,0.448226,0.002121,0.790938
224,10_04_25_questions,ANN,rank_hit@3,0.084850,0.001016,0.150019
225,10_04_25_questions,ANN,rank_hit@5,0.104823,0.001023,0.185012
226,10_04_25_questions,ANN,rank_hit@10,0.133546,0.000996,0.235652


In [8]:
df_weighted['Method'] = df_weighted['Method'].astype(str) + '_' + df_weighted['Threshold'].astype(str) + '_' + df_weighted['Weight'].astype(str)
df_weighted = df_weighted.drop(columns=['Threshold','Weight'])

KeyError: 'Threshold'

In [9]:
df_weighted

,FileID,Method,Metric,Weighted Mean,95% CI
0,10_04_25_questions,TB_ENN,mrr@3,0.150019,0.001282
1,10_04_25_questions,TB_ENN,mrr@5,0.185012,0.001255
2,10_04_25_questions,TB_ENN,mrr@10,0.235652,0.001145
3,10_04_25_questions,TB_ENN,precision@3,0.080673,0.000609
4,10_04_25_questions,TB_ENN,precision@5,0.079525,0.000418
...,...,...,...,...,...
223,10_04_25_questions,ANN,hit@10,0.448226,0.002121
224,10_04_25_questions,ANN,rank_hit@3,0.084850,0.001016
225,10_04_25_questions,ANN,rank_hit@5,0.104823,0.001023
226,10_04_25_questions,ANN,rank_hit@10,0.133546,0.000996


In [5]:
df_re['Method'] = df_re['retrieval_method'].astype(str) + '_' + df_re['weighted'].astype(str)# + '_' + df_re['thr'].astype(str) + 

# Step 2: Drop the old method and modifiers if you don't need them
df_re = df_re.drop(columns=['retrieval_method', 'thr', 'weighted'])

# Step 3: Melt the dataframe so that metric names become a column, and values are aligned accordingly
df_melted = df_re.melt(id_vars=['Method'], var_name='Metric', value_name='Value')

df_melted

,Method,Metric,Value
0,TB_ENN_True,mrr_3,0.000000
1,TB_ENN_True,mrr_3,0.000000
2,TB_ENN_True,mrr_3,0.500000
3,TB_ENN_True,mrr_3,0.333333
4,TB_ENN_True,mrr_3,0.000000
...,...,...,...
48141739,ANN_True,time,0.009868
48141740,ANN_True,time,0.009908
48141741,ANN_True,time,0.009971
48141742,ANN_True,time,0.010311


In [14]:
def format_weighted_value(row):
    mean = f"{row['Weighted Mean']:.3f}"
    ci = f"{row['95% CI']:.3f}"
    value = f"{mean} \pm {ci}"

    metric = row["Metric"]
    method = row["Method"]

    """
    # Agregar '*' si Kruskal-Wallis indica diferencias significativas
    if metric in significance_dict["Kruskal-Wallis p-value"]:
        kw_p_value = significance_dict["Kruskal-Wallis p-value"][metric]
        if kw_p_value < 0.05:
            value += " *"  # Indica que hay diferencias globales
    """
    significance_marker = ""
    if metric in significance_dict["Significant post-hoc tests (Dunn)"]:
        dunn_df = significance_dict["Significant post-hoc tests (Dunn)"][metric]
        for other_method in dunn_df.index:
            if method in dunn_df.columns and other_method in dunn_df.index:
                p_val = dunn_df.loc[other_method, method]
                if p_val < 0.001:
                    significance_marker = "***"
                elif p_val < 0.01:
                    significance_marker = "**"
                elif p_val < 0.05:
                    significance_marker = "*"

        value += f"^{{{significance_marker}}}"
    
    if row["Weighted Mean"] == row["BestValue"]:
        return f"\\(\\boldsymbol{{{value}}}\\)"
    else:
        return f"\\({value}\\)"

df_long = df_melted
df_weighted['Weighted Mean'] = pd.to_numeric(df_weighted['Weighted Mean'], errors='coerce')
df_weighted['95% CI'] = pd.to_numeric(df_weighted['95% CI'], errors='coerce')
normality_tests = {}
for metric in df_long["Metric"].unique():
    for method in df_long["Method"].unique():
        values = df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"]
        if len(values) > 3:  # Shapiro necesita al menos unos pocos valores
            stat, p_value = stats.shapiro(values)
            normality_tests[(metric, method)] = p_value

# ANOVA or Kruskal-Wallis
anova_results = {}
kruskal_results = {}

for metric in df_long["Metric"].unique():
    groups = [df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"].dropna() for method in df_long["Method"].unique()]
    # if normality in at least one group
    normal = all(p > 0.05 for p in [normality_tests.get((metric, method), 0) for method in df_long["Method"].unique()])
    
    if normal:
        stat, p_value = stats.f_oneway(*groups)
        anova_results[metric] = p_value
    else:
        stat, p_value = stats.kruskal(*groups)
        kruskal_results[metric] = p_value

# post-hoc tests if Kruskal-Wallis is significant
posthoc_results = {}
for metric, p in kruskal_results.items():
    if p < 0.05:  # Si hay diferencias significativas
        df_filtered = df_long[df_long["Metric"] == metric]
        posthoc = sp.posthoc_dunn(df_filtered, val_col="Value", group_col="Method", p_adjust="bonferroni")
        posthoc_results[metric] = posthoc


    significance_dict = {
        "ANOVA p-value": anova_results,
        "Kruskal-Wallis p-value": kruskal_results,
        "Significant post-hoc tests (Dunn)": posthoc_results
    }
    df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
    
    df_weighted_pivot = df_weighted.pivot_table(index=["FileID", "Method"], columns="Metric", values="Weighted Mean ± CI", aggfunc="first")
    
    latex_output = "\\begin{table*}[h]\n\\centering\n\\resizebox{\\textwidth}{!}{%\n\\begin{tabular}{lc" + "c" * len(df_weighted_pivot.columns) + "}\n\\arrayrulecolor{black}\n\\toprule\n"
    
    latex_output += "File & \\textbf{Method} & " + " & ".join([f"\\textbf{{{col}}}" for col in df_weighted_pivot.columns]) + " \\\\\n\\midrule\n"
    
    current_file_id = None 
    for (file_idx, method), row in df_weighted_pivot.iterrows():
        if file_idx != current_file_id:  
            if current_file_id is not None:
                latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"

            col_n = str(len(df_weighted_pivot.columns) + 2)
            file_id = str(file_idx+'1')
            
            latex_output += f"\\multicolumn{{{col_n}}}{{l}}{{\\textbf{{File {file_id}}}}} \\\\\n"
            
            latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"
            
            current_file_id = file_idx  
        
        row_start = "\\rowcolor{tableblue} " if method == "XX" else ""
        
        latex_output += row_start + f"& \\textbf{{{method}}} & " + " & ".join(row.astype(str)) + " \\\\\n"
    
    latex_output += "\\bottomrule\n\\end{tabular}}\n\\caption{Performance Metrics per Method for Each File (Best values in bold)}\n\\label{tab:grouped_results}\n\\end{table*}"
    
    print(latex_output)


<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_3395317/2029479833.py:4: SyntaxWarning: invalid escape sequence '\p'
  value = f"{mean} \pm {ci}"
/export/usuarios_ml4ds/ammesa/TFG-LLMs/.venv_idx/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 422296.
  res = hypotest_fun_out(*samples, **kwds)


\begin{table*}[h]
\centering
\resizebox{\textwidth}{!}{%
\begin{tabular}{lcccccccccccccccccccc}
\arrayrulecolor{black}
\toprule
File & \textbf{Method} & \textbf{hit@10} & \textbf{hit@3} & \textbf{hit@5} & \textbf{mrr@10} & \textbf{mrr@3} & \textbf{mrr@5} & \textbf{ndcg@10} & \textbf{ndcg@3} & \textbf{ndcg@5} & \textbf{precision@10} & \textbf{precision@3} & \textbf{precision@5} & \textbf{rank_hit@10} & \textbf{rank_hit@3} & \textbf{rank_hit@5} & \textbf{recall@10} & \textbf{recall@3} & \textbf{recall@5} & \textbf{time} \\
\midrule
\multicolumn{21}{l}{\textbf{File 10_04_25_questions1}} \\
\arrayrulecolor{black}\specialrule{0.5pt}{0pt}{0pt}\arrayrulecolor{black}
& \textbf{ANN} & \(0.448 \pm 0.002\) & \(0.137 \pm 0.001\) & \(0.225 \pm 0.002\) & \(0.134 \pm 0.001\) & \(0.085 \pm 0.001\) & \(0.105 \pm 0.001\) & \(0.205 \pm 0.001\) & \(0.098 \pm 0.001\) & \(0.134 \pm 0.001\) & \(0.045 \pm 0.000\) & \(0.046 \pm 0.000\) & \(0.045 \pm 0.000\) & \(0.134 \pm 0.001\) & \(0.085 \pm 0.001\) & \(0.105

In [15]:
significance_dict['Significant post-hoc tests (Dunn)']

{'mrr_3':               ANN_True  ENN_True  TB_ANN_False   TB_ANN_True  TB_ENN_False  \
 ANN_True           1.0  0.000000  0.000000e+00  0.000000e+00      0.000000   
 ENN_True           0.0  1.000000  0.000000e+00  0.000000e+00      1.000000   
 TB_ANN_False       0.0  0.000000  1.000000e+00  9.438081e-09      0.000000   
 TB_ANN_True        0.0  0.000000  9.438081e-09  1.000000e+00      0.000000   
 TB_ENN_False       0.0  1.000000  0.000000e+00  0.000000e+00      1.000000   
 TB_ENN_True        0.0  0.023097  0.000000e+00  0.000000e+00      0.820387   
 
               TB_ENN_True  
 ANN_True         0.000000  
 ENN_True         0.023097  
 TB_ANN_False     0.000000  
 TB_ANN_True      0.000000  
 TB_ENN_False     0.820387  
 TB_ENN_True      1.000000  ,
 'precision_3':               ANN_True  ENN_True  TB_ANN_False   TB_ANN_True  TB_ENN_False  \
 ANN_True           1.0  0.000000  0.000000e+00  0.000000e+00      0.000000   
 ENN_True           0.0  1.000000  0.000000e+00  0.000000e+